In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.datasets import make_regression
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn import metrics # 모델평가시 이용

In [5]:
# dataset 형성하기
X, y = make_regression(n_samples=2000, # sample 의 수는 2000개
                       n_features=20, # x 변수는 20개
                       n_informative=4, # 그중 y 변수와 관련되는 변수는 4개  
                       noise = 3       # nosie 
                      ,effective_rank=15 #  x 변수중 서로 독립인 수 (5개는 서로 관련)
                      ,tail_strength=0.3 # 관련된 변수의 관련성
                      ,random_state=0) # random state 고정
X = pd.DataFrame(X)
X[[1,4,6,10,13]]=X[[1,4,6,10,13]]*-1

In [6]:
# dataset train/test set 으로 나누기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

# Random Forest (회귀)

## 파라미터 범위 설정

In [7]:
bayes_params = {
    'min_samples_split' : (2,100),
    'min_samples_leaf' : (1,100),
    'max_depth': (1, 10),}

In [8]:
from sklearn.ensemble import RandomForestRegressor
rf_model=RandomForestRegressor()

## 평가 함수 정의

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import numpy as np

# 우선 우리가 tuning 하고 싶은 값들을 받고, 그에따라서 tuning 이 되게 조절해봅시다.
# 우선 max_depth, min_samples_split, min_samples_leaf 를 이용한다고 해 봅시다.
def ran_mse_eval(max_depth,
                min_samples_split,
                min_samples_leaf,):
    # 여기에서는 우리가 조절해야할 파라미터들을 dic 형태로 정의합니다.
    params = {
        "n_estimators": 100 , # 굳이 parameter grid 로 찾고싶지는 않지만 ,default 값이 아니라 다른값을 주고싶을떄 이렇게 200으로 고정해서 하고싶다고 합시다!
        'max_depth': int(round(max_depth)), # 이 때에 정수값이 들어가야하는 경우가 있는데요, 그 떄에는 int(round 로 처리를 해주어야 합니다.
        'min_samples_split' : int(round(min_samples_split)),
        'min_samples_leaf' : int(round(min_samples_leaf))}
    print("params:", params)  # 어떤 파라미터를 사용하였는지 print 하게 해서 학습과정을 지켜보도록 해요~
    rf_model = RandomForestRegressor(**params) # 우리의 모델을 정의합니다.
    
    # 여기서 result 를 뽑아내기 위해서는 2가지 경우가 있습니다! 
    
    # 1. y_val/ x_val 을 나누어서 평가하는 경우
    #rf_model.fit(X_train, y_train) # 먼저 train 에 학습시킨 후
    #valid_proba = rf_model.predict(X_val) # X_val 에 대해서 예측하고
    #result = -1 * mean_squared_error(y_val, valid_proba) # 그에 따른 mse 스코어를 구합니다.
    
    # 하지만 위 같은 경우.. 뭔가 결과가 y_val/ x_val 에 과적합될 수도 있을거같아서 
    # 2. 그냥 X_train/ y_train 의 cv 값을 하게되면 더 좋아보인다! 
    cv_value = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error') # 이떄 scoring 은 주최측에서 정한 값을 넣어야 게엣죠오?
    result = np.mean(cv_value) # cv_value 는 list 형태로 나오게 되니까!
    return result # 이 result 값이 커지게 베이지안optimization 이 학습하게 됩니다.

    print('mse:', result)  # 그 값을 도출 print 해서 잘 학습하고 있는지 (줄여지는 방향으로) 알 아 보아요~

## Optimization

BayesianOptimization 객체를 생성합니다. 
이때 생성 인자로 앞에서 만든 평가함수 lgb_roc_eval 함수와 튜닝할 하이퍼 파라미터의 범위값을 설정한 딕셔너리 변수인 bayes_params를 입력합니다.

In [10]:
from bayes_opt import BayesianOptimization
# 객체를 형성한다
BO_rf = BayesianOptimization(ran_mse_eval, bayes_params, random_state=0)

이제 입력받은 평가함수에 튜닝할 하이퍼 파라미터의 값을 반복적으로 입력하여 최적 하이퍼 파라미터를 튜닝할 준비가 되었습니다. 
BayesianOptimization객체에서 maximize()메소드를 호출하면 이를 수행할 수 있습니다. 

In [11]:
BO_rf.maximize(init_points=5, n_iter=100) 
# 우리의 object function 을 maximize! 하려 한다. 그래서 위에서 return 을 negative mse 를 쓴 것이다.
# 최적 파라미터 도출 작업을 n_iter 만큼 반복하여 수행합니다!
# init_points = 첫 시작지점의 score 를 5개 돌려서 알아보는것.

|   iter    |  target   | max_depth | min_sa... | min_sa... |
-------------------------------------------------------------
params: {'n_estimators': 100, 'max_depth': 6, 'min_samples_split': 61, 'min_samples_leaf': 72}
|  1        | -9.077    |  5.939    |  71.8     |  61.07    |
params: {'n_estimators': 100, 'max_depth': 6, 'min_samples_split': 65, 'min_samples_leaf': 43}
|  2        | -8.963    |  5.904    |  42.94    |  65.3     |
params: {'n_estimators': 100, 'max_depth': 5, 'min_samples_split': 96, 'min_samples_leaf': 89}
|  3        | -9.172    |  4.938    |  89.29    |  96.44    |
params: {'n_estimators': 100, 'max_depth': 4, 'min_samples_split': 54, 'min_samples_leaf': 79}
|  4        | -9.102    |  4.451    |  79.38    |  53.83    |
params: {'n_estimators': 100, 'max_depth': 6, 'min_samples_split': 9, 'min_samples_leaf': 93}
|  5        | -9.199    |  6.112    |  92.63    |  8.962    |
params: {'n_estimators': 100, 'max_depth': 8, 'min_samples_split': 61, 'min_samples_leaf': 1

params: {'n_estimators': 100, 'max_depth': 10, 'min_samples_split': 74, 'min_samples_leaf': 41}
|  42       | -8.959    |  9.924    |  40.68    |  73.62    |
params: {'n_estimators': 100, 'max_depth': 10, 'min_samples_split': 20, 'min_samples_leaf': 46}
|  43       | -9.023    |  10.0     |  46.11    |  20.01    |
params: {'n_estimators': 100, 'max_depth': 1, 'min_samples_split': 91, 'min_samples_leaf': 50}
|  44       | -9.967    |  1.0      |  50.1     |  91.09    |
params: {'n_estimators': 100, 'max_depth': 10, 'min_samples_split': 98, 'min_samples_leaf': 29}
|  45       | -8.916    |  9.976    |  28.72    |  98.46    |
params: {'n_estimators': 100, 'max_depth': 10, 'min_samples_split': 87, 'min_samples_leaf': 20}
|  46       | -8.887    |  9.546    |  20.11    |  86.72    |
params: {'n_estimators': 100, 'max_depth': 2, 'min_samples_split': 92, 'min_samples_leaf': 26}
|  47       | -9.37     |  1.537    |  25.59    |  92.25    |
params: {'n_estimators': 100, 'max_depth': 10, 'min_sa

|  83       | -9.071    |  9.563    |  70.14    |  53.28    |
params: {'n_estimators': 100, 'max_depth': 10, 'min_samples_split': 64, 'min_samples_leaf': 1}
|  84       | -8.849    |  10.0     |  1.0      |  64.31    |
params: {'n_estimators': 100, 'max_depth': 10, 'min_samples_split': 46, 'min_samples_leaf': 81}
|  85       | -9.124    |  9.926    |  81.11    |  45.87    |
params: {'n_estimators': 100, 'max_depth': 10, 'min_samples_split': 21, 'min_samples_leaf': 27}
|  86       | -8.923    |  10.0     |  26.59    |  20.79    |
params: {'n_estimators': 100, 'max_depth': 10, 'min_samples_split': 81, 'min_samples_leaf': 60}
|  87       | -9.047    |  10.0     |  59.91    |  80.96    |
params: {'n_estimators': 100, 'max_depth': 10, 'min_samples_split': 91, 'min_samples_leaf': 13}
|  88       | -8.866    |  10.0     |  12.9     |  91.5     |
params: {'n_estimators': 100, 'max_depth': 10, 'min_samples_split': 63, 'min_samples_leaf': 19}
|  89       | -8.816    |  9.816    |  18.83    |  63

In [12]:
# BayesianOptimization 객체의 res 속성은 하이퍼 파라미터 튜닝을 하는 과정에서의 metric 값과 그때의 하이퍼 파라미터 값을 가지고 있음. 

In [13]:
#BO_rf.res 

BayesianOptimization 객체의 max 속성은 최고 높은 성능 Metric를 가질때의 하이퍼 파라미터 값을 가지고 있음.

In [14]:
BO_rf.max
# 이떄에 우리는 아래 값들을 반올림 해서(int(round)) 사용했음을 기억하세요!
# 즉 max depth = 5 , min_sample_leaf = 3 , min_samples_split = 6 이 됩니다. # 이는 실행마다 달라지니까 주의하세용

{'target': -8.784110903964962,
 'params': {'max_depth': 10.0,
  'min_samples_leaf': 1.0,
  'min_samples_split': 42.88383890095772}}

## 최종 평가!

In [15]:
from sklearn.metrics import mean_squared_error

max_params = BO_rf.max['params']
max_params['min_samples_leaf'] = int(round(max_params['min_samples_leaf']))
max_params['min_samples_split'] = int(round(max_params['min_samples_split']))
model_rf = RandomForestRegressor(n_estimators=100, **max_params)
model_rf.fit(X_train, y_train)
y_pred = model_rf.predict(X_test)
score = mean_squared_error(y_pred,y_test)

print('mse score : {0:.4f}'.format(score))

mse score : 9.0501


## 비교 모델

확실히... 그냥 search 보다는 우수한 성능을 보여주고있네요.

In [16]:
from sklearn.ensemble import RandomForestRegressor
model =  RandomForestRegressor(n_estimators=100)
# scoring 은 default 이므로 model 의 자체 scoring 으로 들어간다. 
# n_estimator = 500 클수록 좋으나 내 컴퓨터가 버티질 못할듯.
model.fit(X_train,y_train)

RandomForestRegressor()

In [17]:
y_pred = model.predict(X_test)
print ("MSE :", metrics.mean_squared_error(y_test, y_pred))
print('R_squared :',model.score(X_test, y_test)) 

MSE : 9.270738756666542
R_squared : 0.13994842989823375


# Light LGB ( 분류 )

In [44]:
from sklearn.datasets import fetch_covtype
covtype = fetch_covtype()

In [45]:
X = pd.DataFrame(covtype.data, 
                  columns=["x{:02d}".format(i + 1) for i in range(covtype.data.shape[1])],
                  dtype=int)
y = covtype.target

In [46]:
X

,x01,x02,x03,x04,x05,x06,x07,x08,x09,x10,...,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,0
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,0
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,0
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,0
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,...,0,0,0,0,0,0,0,0,0,0
581008,2391,152,19,67,12,95,240,237,119,845,...,0,0,0,0,0,0,0,0,0,0
581009,2386,159,17,60,7,90,236,241,130,854,...,0,0,0,0,0,0,0,0,0,0
581010,2384,170,15,60,5,90,230,245,143,864,...,0,0,0,0,0,0,0,0,0,0


In [51]:
X = X.iloc[0:5000,:]
y = y[0:5000]

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

## 파라미터 범위 설정

In [55]:
bayes_params = {
    'num_leaves': (24, 45), # 범위값으로 인식하게 된다!
    'colsample_bytree':(0.5, 1),  
    'subsample': (0.5, 1),
    'max_depth': (4, 12),
    'reg_alpha': (0, 0.5),
    'reg_lambda': (0, 0.5), 
    'min_split_gain': (0.001, 0.1),
    'min_child_weight':(5, 50)
}

테스트 해볼 하이퍼 파라미터의 범위 값을 설정하였으면 BaysianOptimization에서 호출하여 모델을 최적화하는 함수를 만들어 보겠습니다.

해당 함수는 BaysianOptimization에서 하이퍼 파라미터를 튜닝하기 위해 호출되면 제대로 튜닝이 되고 있는지를 판단하기 위해서 모델을 학습/평가하고 이에 따른 평가 지표를 반환하는 형식으로 만들어집니다. 이 평가 함수는 BayesianOptimization 객체에서 파라미터를 변경하면서 호출되므로 함수의 인자로 앞에서 딕셔너리로 설정된 파라미터들을 가지게 됩니다.  

## 평가 함수 정의

In [84]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

def lgb_acc_eval(num_leaves, 
                 colsample_bytree,
                 subsample, 
                 max_depth, 
                 reg_alpha, 
                 reg_lambda, 
                 min_split_gain, 
                 min_child_weight):
    # 하이퍼 파라미터를 튜닝하기 위해 , 이게 제대로! 학습이 되고있는지 판단하기 위해 모델을 학습/ 평가하고 이에 따른 평가 지표를 반환하는 형식이 된다.
    params = {
        "n_estimator":200,
        "learning_rate":0.02,
        'num_leaves': int(round(num_leaves)), # 이 값은 정수형을 return 받아야 하므로! round/ int 를 차례로 받는다.
        'colsample_bytree': colsample_bytree, 
        'subsample': subsample,
        'max_depth': int(round(max_depth)),
        'reg_alpha': reg_alpha,
        'reg_lambda': reg_lambda, 
        'min_split_gain': min_split_gain,
        'min_child_weight': min_child_weight,
        'verbosity': -1
    }
    print("params:", params)  # 어 떤 파라미터를 사용하였는지
    lgb_model = LGBMClassifier(**params) # 모델! 
    cv_value = cross_val_score(lgb_model, X_train, y_train, cv=5, scoring='accuracy')
    
    result = np.mean(cv_value) # cv_value 는 list 형태로 나오게 되니까!
    return result # 이 result 값이 커지게 베이지안optimization 이 학습하게 됩니다.

    print('accuracy :', result)  # 그 값을 도출 print 해서 잘 학습하고 있는지 (줄여지는 방향으로) 알 아 보아요~    return roc_preds

## Optimization

In [85]:
from bayes_opt import BayesianOptimization
# 객체를 형성한다
BO_lgb = BayesianOptimization(lgb_acc_eval, bayes_params, random_state=0)

In [86]:
BO_lgb.maximize(init_points=5, n_iter=10)

|   iter    |  target   | colsam... | max_depth | min_ch... | min_sp... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------
params: {'n_estimator': 200, 'learning_rate': 0.02, 'num_leaves': 33, 'colsample_bytree': 0.7744067519636624, 'subsample': 0.9458865003910399, 'max_depth': 10, 'reg_alpha': 0.32294705653332806, 'reg_lambda': 0.21879360563134626, 'min_split_gain': 0.05494343511669279, 'min_child_weight': 32.12435192322397, 'verbosity': -1}
|  1        |  0.7729   |  0.7744   |  9.722    |  32.12    |  0.05494  |  32.9     |  0.3229   |  0.2188   |  0.9459   |
params: {'n_estimator': 200, 'learning_rate': 0.02, 'num_leaves': 36, 'colsample_bytree': 0.9818313802505146, 'subsample': 0.5435646498507704, 'max_depth': 7, 'reg_alpha': 0.4627983191463305, 'reg_lambda': 0.03551802909894347, 'min_split_gain': 0.05336059705553755, 'min_child_weight': 40.627626713719906, 'ver

In [87]:
# BayesianOptimization 객체의 res 속성은 하이퍼 파라미터 튜닝을 하는 과정에서의 metric 값과 그때의 하이퍼 파라미터 값을 가지고 있음. 

In [88]:
BO_lgb.res

[{'target': 0.7728571428571429,
  'params': {'colsample_bytree': 0.7744067519636624,
   'max_depth': 9.721514930979357,
   'min_child_weight': 32.12435192322397,
   'min_split_gain': 0.05494343511669279,
   'num_leaves': 32.896750786116996,
   'reg_alpha': 0.32294705653332806,
   'reg_lambda': 0.21879360563134626,
   'subsample': 0.9458865003910399}},
 {'target': 0.7648571428571429,
  'params': {'colsample_bytree': 0.9818313802505146,
   'max_depth': 7.067532150606222,
   'min_child_weight': 40.627626713719906,
   'min_split_gain': 0.05336059705553755,
   'num_leaves': 35.92893578297258,
   'reg_alpha': 0.4627983191463305,
   'reg_lambda': 0.03551802909894347,
   'subsample': 0.5435646498507704}},
 {'target': 0.7594285714285715,
  'params': {'colsample_bytree': 0.5101091987201629,
   'max_depth': 10.660958764383505,
   'min_child_weight': 40.01705379274327,
   'min_split_gain': 0.08713120267643511,
   'num_leaves': 44.55098518688804,
   'reg_alpha': 0.3995792821083618,
   'reg_lambda':

BayesianOptimization 객체의 max 속성은 최고 높은 성능 Metric를 가질때의 하이퍼 파라미터 값을 가지고 있음.

In [79]:
BO_lgb.max

{'target': 0.8274285714285714,
 'params': {'colsample_bytree': 0.5,
  'max_depth': 12.0,
  'min_child_weight': 6.153818666923531,
  'min_split_gain': 0.1,
  'num_leaves': 45.0,
  'reg_alpha': 0.0,
  'reg_lambda': 0.0,
  'subsample': 1.0}}

## 최종 평가

In [90]:
from sklearn.metrics import accuracy_score

max_params = BO_rf.max['params']
max_params['min_samples_leaf'] = int(round(max_params['min_samples_leaf']))
max_params['min_samples_split'] = int(round(max_params['min_samples_split']))
max_params['max_depth'] = int(max_params['max_depth'])
lgb_model = LGBMClassifier(n_estimators=200,learning_rate=0.2, **max_params)
lgb_model.fit(X_train, y_train) # train 을 전체로 해서 더 높은가바..
y_pred = lgb_model.predict(X_test)
score = accuracy_score(y_pred,y_test)

print('accuracy : {0:.4f}'.format(score))

mse score : 0.8713
